In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggleb

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jhoward/lsun_bedroom

100% 8.89G/8.89G [05:01<00:00, 42.8MB/s]
100% 8.89G/8.89G [05:01<00:00, 31.7MB/s]


In [ ]:
!unzip lsun_bedroom.zip

In [1]:
!git clone https://github.com/mikolajzalewski/Deep_Learning.git

Cloning into 'Deep_Learning'...
remote: Enumerating objects: 51101, done.
remote: Counting objects: 100% (1091/1091), done.
remote: Compressing objects: 100% (486/486), done.
remote: Total 51101 (delta 607), reused 986 (delta 585), pack-reused 50010
Receiving objects: 100% (51101/51101), 768.10 MiB | 27.28 MiB/s, done.
Resolving deltas: 100% (608/608), done.
Updating files: 100% (128/128), done.


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [25]:
%cd /content/gdrive/MyDrive/Colab Notebooks/data0/lsun/bedroom

/content/gdrive/MyDrive/Colab Notebooks/data0/lsun/bedroom


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import os
import cv2
import numpy as np
import pandas as pd

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from gan_package.gan import GAN
from gan_package.vanillaGAN import VanillaGAN_Generator, VanillaGAN_Discriminator
from gan_package.dcGAN import DCGAN_Generator, DCGAN_Discriminator

In [6]:
image_size = 256
batch_size = 128
workers = 2
root = 'D:/deep/data0/lsun/bedroom/0/0'

In [8]:
import torch

def frecher_chet_distance(real_images, fake_images, discriminator):
    real_images_outputs = discriminator(real_images)
    fake_images_outputs = discriminator(fake_images)

    real_images_outputs = real_images_outputs.view(-1)
    fake_images_outputs = fake_images_outputs.view(-1)

    return torch.mean(fake_images_outputs) - torch.mean(real_images_outputs)

In [9]:
dataset = dset.ImageFolder(root=root,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

results = pd.DataFrame(columns=['numb_of_epochs', 'batch_size', 'learning_rate', 'vanilla_gan_dif', 'dc_gan_dif'])

for numb_of_epochs in [1, 200, 300]:
    for batch_size in [64, 128, 256]:
        for learning_rate in [0.0001, 0.001, 0.01]:
            # Create the dataloader
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    shuffle=True, num_workers=workers)
            # Vanilla GAN
            IMG_SHAPE = dataloader.dataset[0][0].shape
            N_OUT = torch.prod(torch.tensor(IMG_SHAPE))
            LATENT_DIM = 100
            vanilla_generator = VanillaGAN_Generator(latent_dim=LATENT_DIM, img_shape=IMG_SHAPE, n_out=N_OUT)
            vanilla_discriminator = VanillaGAN_Discriminator(img_shape=IMG_SHAPE)
            vanilla_gan = GAN(generator=vanilla_generator, discriminator=vanilla_discriminator)

            criterion = nn.BCELoss()

            vanilla_generator_optimizer = optim.Adam(vanilla_generator.parameters(), lr=learning_rate)
            vanilla_discriminator_optimizer = optim.Adam(vanilla_discriminator.parameters(), lr=learning_rate)
            vanilla_gan.train(dataloader=dataloader,
                            discriminator_optimizer=vanilla_discriminator_optimizer,
                            generator_optimizer=vanilla_generator_optimizer,
                            criterion=criterion,
                            num_epochs=numb_of_epochs)
            print('Training finished')

            vanilla_discriminator_dif = frecher_chet_distance(dataloader.dataset[0][0].unsqueeze(0), vanilla_gan.generate_samples(1), vanilla_discriminator)
            

            # DC-GAN
            dc_generator = DCGAN_Generator(latent_dim=LATENT_DIM, feature_maps_size=64, num_channels=3)
            dc_discriminator = DCGAN_Discriminator(num_channels=3, feature_maps_size=64, n_out=1)  
            dc_gan = GAN(generator=dc_generator, discriminator=dc_discriminator)

            criterion = nn.BCELoss()

            dc_generator_optimizer = optim.Adam(dc_generator.parameters(), lr=learning_rate)
            dc_discriminator_optimizer = optim.Adam(dc_discriminator.parameters(), lr=learning_rate)
            dc_gan.train(dataloader=dataloader,
                        discriminator_optimizer=dc_discriminator_optimizer,
                        generator_optimizer=dc_generator_optimizer,
                        criterion=criterion,
                        num_epochs=numb_of_epochs)
            
            dc_gan_dif = frecher_chet_distance(dataloader.dataset[0][0].unsqueeze(0), dc_gan.generate_samples(1), dc_discriminator)

            results = pd.concat([results, pd.DataFrame([[numb_of_epochs, batch_size, learning_rate, vanilla_discriminator_dif, dc_gan_dif]], columns=['numb_of_epochs', 'batch_size', 'learning_rate', 'vanilla_gan_dif', 'dc_gan_dif'])])

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 19/19 [01:35<00:00,  5.02s/it]

Epoch 1/1: Generator loss: 0.0009317326032562761 Discriminator loss: 0.0009445386217129941
Training finished


AttributeError: 'GAN' object has no attribute 'generate_samples'